In [1]:
import pandas as pd

# Load the datasets
file_path_internet = '/content/API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'
file_path_poverty = '/content/API_SI.POV.DDAY_DS2_en_csv_v2_227712.csv'
file_path_unemployment = '/content/API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_228704.csv'

# Read the files, skipping the initial rows that are not data headers
data_internet = pd.read_csv(file_path_internet, skiprows=4)
data_poverty = pd.read_csv(file_path_poverty, skiprows=4)
data_unemployment = pd.read_csv(file_path_unemployment, skiprows=4)

# Filter for Brazil
brazil_internet = data_internet[data_internet['Country Name'] == 'Brazil']
brazil_poverty = data_poverty[data_poverty['Country Name'] == 'Brazil']
brazil_unemployment = data_unemployment[data_unemployment['Country Name'] == 'Brazil']

# Define common columns for merging
# Define common columns for merging
common_columns = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + \
    [str(year) for year in range(1960, 2024)]  # Including years 1960 to 2023

# Concatenate data vertically, aligning by the specified common columns
brazil_combined = pd.concat([
    brazil_internet[common_columns],
    brazil_poverty[common_columns],
    brazil_unemployment[common_columns]
], axis=0)

# Reset index to make it clean
brazil_combined.reset_index(drop=True, inplace=True)



In [2]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,54.551002,58.327952,60.87254,67.471285,70.434283,73.91244,81.342694,80.689893,80.527751,NaN
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,3.300000,3.900000,4.70000,5.300000,5.300000,5.40000,2.000000,5.800000,3.500000,NaN
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.760000,8.540000,11.58000,12.790000,12.330000,11.94000,13.700000,13.160000,9.230000,7.776


In [3]:
# Paths to the metadata files
metadata_file_internet = '/content/Metadata_Indicator_API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'
metadata_file_poverty = '/content/Metadata_Indicator_API_SI.POV.DDAY_DS2_en_csv_v2_227712.csv'
metadata_file_unemployment = '/content/Metadata_Indicator_API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_228704.csv'

# Load the metadata files
metadata_internet = pd.read_csv(metadata_file_internet)
metadata_poverty = pd.read_csv(metadata_file_poverty)
metadata_unemployment = pd.read_csv(metadata_file_unemployment)

# Select only the relevant columns from each metadata DataFrame
columns_to_keep = ['INDICATOR_CODE', 'SOURCE_NOTE', 'SOURCE_ORGANIZATION']
metadata_internet = metadata_internet[columns_to_keep]
metadata_poverty = metadata_poverty[columns_to_keep]
metadata_unemployment = metadata_unemployment[columns_to_keep]

# Create a single metadata DataFrame by concatenating all metadata DataFrames
metadata_combined = pd.concat([metadata_internet, metadata_poverty, metadata_unemployment], ignore_index=True)

# Merge the combined metadata with the brazil_combined DataFrame on 'Indicator Code'
brazil_combined = brazil_combined.merge(metadata_combined, how='left', left_on='Indicator Code', right_on='INDICATOR_CODE')

# Drop the extra 'INDICATOR_CODE' column if it's redundant
brazil_combined.drop('INDICATOR_CODE', axis=1, inplace=True)



In [4]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,SOURCE_NOTE,SOURCE_ORGANIZATION
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,60.87254,67.471285,70.434283,73.91244,81.342694,80.689893,80.527751,NaN,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,4.70000,5.300000,5.300000,5.40000,2.000000,5.800000,3.500000,NaN,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D..."
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,11.58000,12.790000,12.330000,11.94000,13.700000,13.160000,9.230000,7.776,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...


In [5]:
metadata_country_path = '/content/Metadata_Country_API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'

In [6]:
metadata_country = pd.read_csv(metadata_country_path)

In [7]:
brazil_metadata = metadata_country[metadata_country['Country Code'] == 'BRA']

In [8]:
brazil_metadata

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
29,BRA,Latin America & Caribbean,Upper middle income,NaN,Brazil,NaN


In [9]:

brazil_region = brazil_metadata['Region'].values[0] if not brazil_metadata['Region'].empty else 'Not Available'
brazil_income_group = brazil_metadata['IncomeGroup'].values[0] if not brazil_metadata['IncomeGroup'].empty else 'Not Available'


In [10]:
brazil_combined['Region'] = brazil_region
brazil_combined['IncomeGroup'] = brazil_income_group

In [11]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2018,2019,2020,2021,2022,2023,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,70.434283,73.91244,81.342694,80.689893,80.527751,NaN,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,5.300000,5.40000,2.000000,5.800000,3.500000,NaN,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.330000,11.94000,13.700000,13.160000,9.230000,7.776,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income


In [12]:
year_columns = [col for col in brazil_combined.columns if col.isdigit()]

# Specify the ID variables for the melt function
id_vars = [col for col in brazil_combined.columns if col not in year_columns]

# Melt the DataFrame to have years as rows
brazil_melted = pd.melt(brazil_combined, id_vars=id_vars, value_vars=year_columns,
                        var_name='Year', value_name='Value')

# Optionally, reorder the columns for better readability
brazil_melted = brazil_melted[['Year', 'Country Name', 'Indicator Name', 'Indicator Code',
                               'SOURCE_NOTE', 'SOURCE_ORGANIZATION', 'Region', 'IncomeGroup', 'Value']]



In [13]:
brazil_melted

,Year,Country Name,Indicator Name,Indicator Code,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup,Value
0,1960,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,NaN
1,1960,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,NaN
2,1960,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,NaN
3,1961,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,NaN
4,1961,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,NaN
...,...,...,...,...,...,...,...,...,...
187,2022,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,3.500
188,2022,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,9.230
189,2023,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,NaN
190,2023,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,NaN


In [14]:
import pandas as pd

# Assuming 'brazil_melted' is already loaded and transformed as required
# Define a function to determine the category based on the indicator name
def determine_category(indicator_name):
    if "Internet" in indicator_name:
        return "Internet users"
    elif "Unemployment" in indicator_name:
        return "Unemployment"
    elif "Poverty" in indicator_name:
        return "Poverty"
    else:
        return "Other"  # Default category if none of the keywords match

# Apply the function to create the new 'category' column
brazil_melted['category'] = brazil_melted['Indicator Name'].apply(determine_category)

# Optionally, check the distribution of categories to ensure the mapping is correct
print(brazil_melted['category'].value_counts())




category
Internet users    64
Poverty           64
Unemployment      64
Name: count, dtype: int64


In [23]:
brazil_melted.fillna("Data not available",inplace=True)

In [17]:
organization_dict = {
    "Internet users": "International Telecommunication Union",
    "Unemployment": "International Labour Organization",
    "Poverty": "World Bank, Poverty and Inequality Platform"
}


In [18]:
def determine_category(indicator_name):
    if "Internet" in indicator_name:
        return "Internet users"
    elif "Unemployment" in indicator_name:
        return "Unemployment"
    elif "Poverty" in indicator_name:
        return "Poverty"
    else:
        return "Other"

# Apply the function to create the 'Category' column
brazil_melted['Category'] = brazil_melted['Indicator Name'].apply(determine_category)


In [19]:
# Map the 'Category' to 'Organization' using the dictionary
brazil_melted['Organization'] = brazil_melted['Category'].map(organization_dict)


In [24]:
# Print the first few rows to check the new 'Organization' column along with the 'Category'
brazil_melted


,Year,Country Name,Indicator Name,Indicator Code,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup,Value,category,Category,Organization
0,1960,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data not available,Internet users,Internet users,International Telecommunication Union
1,1960,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,Data not available,Poverty,Poverty,"World Bank, Poverty and Inequality Platform"
2,1960,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,Data not available,Unemployment,Unemployment,International Labour Organization
3,1961,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data not available,Internet users,Internet users,International Telecommunication Union
4,1961,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,Data not available,Poverty,Poverty,"World Bank, Poverty and Inequality Platform"
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2022,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,3.5,Poverty,Poverty,"World Bank, Poverty and Inequality Platform"
188,2022,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,9.23,Unemployment,Unemployment,International Labour Organization
189,2023,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data not available,Internet users,Internet users,International Telecommunication Union
190,2023,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,Data not available,Poverty,Poverty,"World Bank, Poverty and Inequality Platform"
